In [ ]:
pip install crewai

In [21]:
from langchain import hub
from langchain.agents import AgentExecutor, tool
from langchain_google_vertexai import VertexAI
import json
import os
from crewai import Agent, Task, Crew, Process

In [22]:
#load json file 
with open('../keys.json') as f:
    keys = json.load(f)

In [23]:
llm = VertexAI(
    model_name='gemini-pro',
    temperature=0,
    top_k=1,
)

In [24]:
llm.invoke("hello world?")

'Hello World!\n\nThis is a very common program that is often used to test if a programming language is working properly. It simply prints the message "Hello World!" to the console.\n\nHere is an example of how to write a "Hello World!" program in Python:\n\n```python\nprint("Hello World!")\n```\n\nYou can run this program by opening a terminal window and typing the following command:\n\n```\npython hello_world.py\n```\n\nThis will print the message "Hello World!" to the console.\n\nHere are some other examples of "Hello World!" programs in different programming languages:\n\n* **C++:**\n\n```c++\n#include <iostream>\n\nint main() {\n  std::cout << "Hello World!" << std::endl;\n  return 0;\n}\n```\n\n* **Java:**\n\n```java\npublic class HelloWorld {\n\n  public static void main(String[] args) {\n    System.out.println("Hello World!");\n  }\n}\n```\n\n* **JavaScript:**\n\n```javascript\nconsole.log("Hello World!");\n```\n\n* **PHP:**\n\n```php\n<?php\n  echo "Hello World!";\n?>\n```\n\n*

In [27]:
# Define your agents with roles and goals
coder = Agent(
  role='coder',
  goal='extract the count per pack from the product title and provide it as JSON',
  backstory="""you are an expert data analyst your job is to extract """,
  verbose=True,
  allow_delegation=False,
  llm=llm # was defined above in the file
)

manager = Agent(
  role='manager',
  goal='review the json provided by the coder and verify that it is correct',
  backstory="""you are a manager, your coder is going to provide you with a final answer in the form of a json string that should the brand name and the count per pack, review it and if it isn't correct ask the coder to do it again""",
  verbose=True,
  allow_delegation=True,
  llm=llm
)

# Create tasks for your agents
task1 = Task(
  description="""extract the json from this product title
  product title: coca cola 12 oz cans (6 pack). you do not need a tool""",
  agent=coder
)

task2 = Task(
  description="""review the final answer provided by the coder, if is incorrect ask the coder to redo it""",
  agent=manager
)

# Instantiate your crew with a sequential process
tech_crew = Crew(
  agents=[coder, manager],
  tasks=[task1 , task2],
  process=Process.sequential  # Tasks will be executed one after the other
)

In [28]:
tech_crew.kickoff()



> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: {
  "countPerPack": 6
}

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: The json string is incomplete. It should include the brand name. Please provide the complete json string.

> Finished chain.


'The json string is incomplete. It should include the brand name. Please provide the complete json string.'